In [10]:
import pandas as pd
import csv
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore")

In [11]:
# reading the CSV file
passengers = pd.read_csv('train.csv')
 
# displaying the contents of the CSV file
# print(passengers.columns)
passengers = passengers[['PassengerId', 'Pclass', 
'Name', 'Sex', 'Age', 'SibSp','Parch', 'Ticket', 
'Fare', 'Cabin', 'Embarked','Survived']]

passengers = passengers.drop(columns=['PassengerId','Name', 'Ticket','Cabin'])

# print(passengers)

In [12]:
X_train = passengers.iloc[:,0:-1]
y_train = passengers.iloc[:,-1]
categorical_cols = ['Sex','Pclass','Embarked']
#import pandas as pd
X_train = pd.get_dummies(X_train, columns = categorical_cols)
# print(np.mean(X_train['Age']))

for i in range(len(X_train['Age'])):
    if np.isnan(X_train['Age'][i]):
        X_train['Age'][i] = np.mean(X_train['Age'])
        

In [13]:
xg_class = xgb.XGBClassifier(objective ='reg:linear', colsample_bytree = 0.2, learning_rate = 0.0000007,
                max_depth = 5, alpha = 10, n_estimators = 10)
xg_class.fit(X_train,y_train)

preds_train = xg_class.predict(X_train)
sum(preds_train == y_train) / len(y_train)

[08:14:29] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-37/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


0.7295173961840629

In [14]:
mlp_class = MLPClassifier( learning_rate='adaptive', max_iter=1000, hidden_layer_sizes=(64, 64))
mlp_class.fit(X_train,y_train)
print("MLP Classifier Score: ",round(mlp_class.score(X_train,y_train) * 100,2), '%')

MLP Classifier Score:  82.6 %


In [15]:
#### TEST SET
# reading the CSV file
test = pd.read_csv('test.csv')
pID = test.iloc[:,0]
test = test.drop(columns=['PassengerId', 'Name', 'Ticket','Cabin'])
test = pd.get_dummies(test, columns = categorical_cols)
# print(test.info)

for i in range(len(test['Age'])):
    if np.isnan(test['Age'][i]):
        test['Age'][i] = np.mean(test['Age'])
    if np.isnan(test['Fare'][i]):
        test['Fare'][i] = np.mean(test['Fare'])


preds_test = mlp_class.predict(test)


In [16]:
submission = np.column_stack((np.array(pID), np.array(preds_test)))
submission = np.row_stack((['PassengerId','Survived'],submission))
with open('submission.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    for i in range(len(submission)):
        writer.writerow(submission[i])




In [17]:
DTC = DecisionTreeClassifier()
DTC.fit(X_train,y_train)
DTC.score(X_train,y_train)
preds_test = DTC.predict(test)